In [ ]:
import os
import pandas as pd

root_directory = '/Users/divya/Documents/Data_collection/final_correct_dataset_with_default//'

dataframes = []

for dirpath, dirnames, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            dataframes.append(df)

big_dataframe = pd.concat(dataframes, ignore_index=True)
big_dataframe

In [ ]:
groups = big_dataframe.groupby('serialnumber')['participant'].first()
groups

In [ ]:
# This block is to select the 2 out of 4 FSRs for all the participants
grouped_df = big_dataframe.groupby('participant')

# Mapping of participant to the columns to copy
participant_column_mapping = {
    1: ['fsr2', 'fsr4'],
    2: ['fsr2', 'fsr4'],
    3: ['fsr2', 'fsr4'],
    4: ['fsr2', 'fsr4'],
    5: ['fsr2', 'fsr4'],
    6: ['fsr2', 'fsr4'],
    9: ['fsr2', 'fsr4'],
    10: ['fsr2', 'fsr3'],
    11: ['fsr2', 'fsr4'],
    12: ['fsr1', 'fsr3'],
    13: ['fsr2', 'fsr4'],
    14: ['fsr2', 'fsr3'],

}

big_dataframe['FSR1'] = ''
big_dataframe['FSR2'] = ''

for participant_id, cols_to_copy in participant_column_mapping.items():
    mask = big_dataframe['participant'] == participant_id
    big_dataframe.loc[mask, 'FSR1'] += big_dataframe.loc[mask, cols_to_copy[0]].astype(str) + ','
    big_dataframe.loc[mask, 'FSR2'] += big_dataframe.loc[mask, cols_to_copy[1]].astype(str) + ','

big_dataframe['FSR1'] = big_dataframe['FSR1'].str.rstrip(',')
big_dataframe['FSR2'] = big_dataframe['FSR2'].str.rstrip(',')

big_dataframe


In [ ]:
big_dataframe['FSR1'] = big_dataframe['FSR2'].astype(float)
big_dataframe['FSR2'] = big_dataframe['FSR2'].astype(float)

In [ ]:
big_dataframe=big_dataframe.drop(columns=['fsr1','fsr2','fsr3','fsr4'])

In [ ]:
import pandas as pd

train_participants = [1, 2, 3, 4, 5, 6, 9, 12, 13]
test_participants = [10, 11, 14]

train_set = big_dataframe[big_dataframe['participant'].isin(train_participants)].copy()
test_set = big_dataframe[big_dataframe['participant'].isin(test_participants)].copy()
big_dataframe =train_set
test_dataframe = test_set
test_dataframe

In [ ]:
y = big_dataframe.groupby('serialnumber')['class'].first()
y_test = test_dataframe.groupby('serialnumber')['class'].first()
y

In [ ]:
X = big_dataframe.drop(columns=['class','orient1','orient2','orient3','participant'])
test = test_dataframe.drop(columns=['class','orient1','orient2','orient3','participant'])
test

In [ ]:
from tsfresh import extract_features, extract_relevant_features
extracted_features = extract_relevant_features(X,y, column_id='serialnumber', column_sort='time')

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

# Number of features you want to retain
num_features_to_retain = 29

# Apply mutual information feature selection
feature_selector = SelectKBest(score_func=mutual_info_regression, k=num_features_to_retain)
selected_features = feature_selector.fit_transform(extracted_features, y)

selected_feature_indices = feature_selector.get_support(indices=True)

# Filter the original feature names to retain only selected features
selected_feature_names = [extracted_features.columns[i] for i in selected_feature_indices]

# Create a DataFrame with the selected features
selected_features_df = extracted_features[selected_feature_names]

In [ ]:
selected_features_df = selected_features_df.drop('gyro3__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"', axis=1)
#selected_features_df = selected_features_df.drop('gyro3__fft_aggregated__aggtype_"skew"', axis=1)

selected_features_df

In [ ]:
f_feature= pd.Index(["FSR1__cid_ce__normalize_False",'FSR1__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0'])

In [ ]:
import re
from tsfresh.feature_extraction import feature_calculators as tsfresh_fc

def generate_code(input_string,sets):
    # Split the string using '__' as the delimiter
    split_parts = re.split("__", input_string)
    
    part1 = split_parts[0]
    part2 = split_parts[1]
    
    optional_params = {}
    
    for part in split_parts[2:]:

        key, value = part.rsplit("_", 1)
        
        if value.isdigit():
            value = int(value)
        elif value.lower() == "true":
            value = True
        elif value.lower() == "false":
            value = False
        #if key == "f_agg":
            # Replace the second underscore with an equal sign in f_agg parameter
          #  value = f'"{value.replace("_", "=")}"'
        optional_params[key] = value
    
    # Create the code snippet
    code = (
        f"{sets}.groupby('serialnumber')['{part1}'].apply(lambda x: "
        f"tsfresh_fc.{part2}(x, {', '.join([f'{key}={value}' for key, value in optional_params.items()])})).reset_index()"
    )
    
    return code

In [ ]:
new_df = pd.DataFrame()
for feature in f_feature:
    
    code = generate_code(feature,sets="X")
    
    # Execute the code and store the result in a DataFrame
    feature_result = eval(code)
    
    # Rename the column to match the feature name
    feature_result.rename(columns={feature_result.columns[-1]: feature}, inplace=True)
    
    # Merge the result into the main DataFrame
    if new_df.empty:
        new_df = feature_result
    else:
        new_df = pd.merge(new_df, feature_result, on='serialnumber', how='inner')
new_df.set_index('serialnumber', inplace=True)

selected_features_df=pd.concat([selected_features_df, new_df], axis=1)
selected_features_df

In [ ]:
selected_features_df.columns
final_features = selected_features_df.columns
final_features = final_features.append(pd.Index(["fsr2__cid_ce__normalize_False"]))
final_features = final_features.append(pd.Index(['fsr2__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0']))
final_features

In [ ]:
result_df = pd.DataFrame()

# Loop through the selected features
for feature in final_features:
    sets="test"
    # Generate the code for the feature
    if feature == 'gyro3__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"':
        continue
        
    if feature == 'gyro3__fft_aggregated__aggtype_"skew"': 
        continue
    code = generate_code(feature,sets)
    # Execute the code and store the result in a DataFrame
    feature_result = eval(code)
    
    # Rename the column to match the feature name
    feature_result.rename(columns={feature_result.columns[-1]: feature}, inplace=True)
    
    # Merge the result into the main DataFrame
    if result_df.empty:
        result_df = feature_result
    else:
        result_df = pd.merge(result_df, feature_result, on='serialnumber', how='inner')

# Set 'serialnumber' as the index of the result DataFrame
result_df.set_index('serialnumber', inplace=True)
result_df

In [ ]:
result_df = result_df.dropna()
y_test = y_test.loc[result_df.index]

In [ ]:
X_train = selected_features_df 
y_train = y
X_test = result_df
y_test = y_test

In [ ]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
print(len(y))
print(len(y_test))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
import numpy as np
import time

# Create the DecisionTreeClassifier
classifier = DecisionTreeClassifier(max_depth=5)

# Perform k-fold cross-validation
groups = big_dataframe.groupby('serialnumber')['participant'].first()
logo = LeaveOneGroupOut()
accuracy_scores = cross_val_score(classifier, X_train, y_train, groups=groups, cv=logo)

# Calculate the average accuracy
average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
classifier.fit(X_train, y_train)

# Calculate the training time
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]]  # Adjust to the instance you want to predict
start_time_single_prediction = time.time()
single_prediction = classifier.predict(single_instance)
# Calculate the single prediction time
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

# Predict on the test set
dt_y_pred = classifier.predict(X_test)

# Generate a classification report and accuracy for the test set

accuracy = accuracy_score(y_test, dt_y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, dt_y_pred)
print("Classification Report:\n", report)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree

# Visualize the decision tree
plt.figure(figsize=(15, 10))  # Adjust the figure size as needed
plot_tree(classifier, filled=True, feature_names=X_train.columns, class_names=True)
plt.show()

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform k-fold cross-validation
groups = big_dataframe.groupby('serialnumber')['participant'].first()
logo = LeaveOneGroupOut()
accuracy_scores = cross_val_score(rf_classifier, X_train, y_train, groups=groups, cv=logo)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()

rf_classifier.fit(X_train, y_train)
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = rf_classifier.predict(single_instance)
# Calculate the single prediction time
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
class_labels = ["Default","Stop", "Speed_Up", "Slow_Down", "Change_Direction", "Repeat", "Reset"]
# Define a mapping from numeric labels to class names
label_mapping = {
    0: "Default",
    1: "Stop",
    2: "Speed_Up",
    3: "Slow_Down",
    4: "Change_Direction",
    5: "Repeat",
    6: "Reset"
}

y_test_mapped = [label_mapping[label] for label in y_test]
y_pred_mapped = [label_mapping[label] for label in y_pred]

conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)

confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df

In [ ]:
#Gradient Boost

from sklearn.ensemble import GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier(n_estimators=92, random_state=42,max_depth=4,learning_rate=0.2)

groups = big_dataframe.groupby('serialnumber')['participant'].first()
logo = LeaveOneGroupOut()
accuracy_scores = cross_val_score(gb_classifier, X_train, y_train, groups=groups, cv=logo)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
gb_classifier.fit(X_train, y_train)
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = gb_classifier.predict(single_instance)
# Calculate the single prediction time
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = gb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)

In [ ]:
# Grid search for gradient boost model
from sklearn.metrics import accuracy_score

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2, 0.3]
}

gb_classifier = GradientBoostingClassifier(random_state=66)

# Perform Grid Search
grid_search = GridSearchCV(estimator=gb_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Use the best model to make predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


In [ ]:
y_test_mapped = [label_mapping[label] for label in y_test]
y_pred_mapped = [label_mapping[label] for label in y_pred]
conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)
confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)
confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df_percent = confusion_df.divide(confusion_df.sum(axis=1), axis=0) * 100
annotations = confusion_df.astype(str) + '\n' + confusion_df_percent.round(2).astype(str) + '%'

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_df, annot=annotations, fmt='', cmap="Blues", cbar_kws={'label': 'Counts'})

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

plt.savefig('conf-gb.png')

plt.show()


In [ ]:
gb_classifier = GradientBoostingClassifier(n_estimators=120, random_state=42,max_depth=4,learning_rate=0.2)
gb_classifier.fit(X_train, y_train)
y_pred = gb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

In [ ]:
#Bagging
from sklearn.ensemble import BaggingClassifier
base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Create a BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

groups = big_dataframe.groupby('serialnumber')['participant'].first()
logo = LeaveOneGroupOut()
accuracy_scores = cross_val_score(bagging_classifier, X_train, y_train, groups=groups, cv=logo)
average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)


start_time = time.time()
bagging_classifier.fit(X_train, y_train)

# Calculate the training time
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]] 
start_time_single_prediction = time.time()
single_prediction = bagging_classifier.predict(single_instance)

# Calculate the single prediction time
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = bagging_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)

In [ ]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier

base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Create a BaggingClassifier
adaboost_classifier = AdaBoostClassifier(base_classifier, n_estimators=50, random_state=42)

# Perform k-fold cross-validation
groups = big_dataframe.groupby('serialnumber')['participant'].first()
logo = LeaveOneGroupOut()
accuracy_scores = cross_val_score(adaboost_classifier, X_train, y_train, groups=groups, cv=logo)

# Calculate the average accuracy
average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)


start_time = time.time()
adaboost_classifier.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = adaboost_classifier.predict(single_instance)
# Calculate the single prediction time
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = adaboost_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)

In [ ]:
#SVM 
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)


# Perform k-fold cross-validation
groups = big_dataframe.groupby('serialnumber')['participant'].first()
logo = LeaveOneGroupOut()
accuracy_scores = cross_val_score(svm_classifier, X_train, y_train, groups=groups, cv=logo)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
svm_classifier.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = svm_classifier.predict(single_instance)
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
#k-nearest neighbours KNN
knn_classifier = KNeighborsClassifier(n_neighbors=4)

groups = big_dataframe.groupby('serialnumber')['participant'].first()
logo = LeaveOneGroupOut()
accuracy_scores = cross_val_score(knn_classifier, X_train, y_train, groups=groups, cv=logo)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
knn_classifier.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = knn_classifier.predict(single_instance)
# Calculate the single prediction time
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = knn_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
# Grid search for gradient boost model

gb_classifier = GradientBoostingClassifier(random_state=42)

# Define the parameter grid you want to search
param_grid = {
    'n_estimators': [50, 75, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=gb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Perform the grid search on your training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
print("Best Parameters:", best_params)
best_estimator.fit(X_train, y_train)

y_pred = best_estimator.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy on the test set:", accuracy)


In [ ]:
# Grid search for Ada boost model

# Create the base classifier
base_classifier = GradientBoostingClassifier(n_estimators=92, random_state=42)
#base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Create the AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(base_classifier, random_state=42)

# Define the parameter grid you want to search
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=adaboost_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
print("Best Parameters:", best_params)
y_pred = best_estimator.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy on the test set:", accuracy)


In [ ]:
#base_classifier = GradientBoostingClassifier(n_estimators=92, random_state=42)
base_classifier = RandomForestClassifier(n_estimators=40, random_state=42)

# Create the AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(base_classifier, learning_rate =0.001, random_state=42)
adaboost_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = adaboost_classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# Grid search for Bagging model

# Create the base classifier
base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Create the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, random_state=42)

# Define the parameter grid you want to search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_samples': [0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=bagging_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Perform the grid search on your training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)

# Use the best estimator to make predictions on the test data
y_pred = best_estimator.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy on the test set:", accuracy)


In [ ]:
#Downloading best model
import joblib
joblib.dump(gb_classifier, 'gb_classifier.joblib')

In [ ]:
# Confusion matrix for best model
best_model = gb_classifier

y_test_mapped = [label_mapping[label] for label in y_test]
y_pred_mapped = [label_mapping[label] for label in y_pred]

conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)

confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df

In [ ]:
conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)
confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df_percent = confusion_df.divide(confusion_df.sum(axis=1), axis=0) * 100
annotations = confusion_df.astype(str) + '\n' + confusion_df_percent.round(2).astype(str) + '%'

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_df, annot=annotations, fmt='', cmap="Blues", cbar_kws={'label': 'Counts'})

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

plt.savefig('conf-gb.png')

plt.show()